# CIFAR models evaluation

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

## Load CIFAR dataset

In [ ]:
builder, ds_info = tfds.load('cifar10',
                             split=["train", "test[:40%]", "test[40%:]"],
                             as_supervised=True, 
                             with_info=True)

In [ ]:
ds_train = builder[0]
ds_validation = builder[1]
ds_test = builder[2]

print('train size: ' + str(len(ds_train)))
print('validation size: ' + str(len(ds_validation)))
print('test size: ' + str(len(ds_test)))

## Load models

In [ ]:
from models import CIFAR_Detectition_Pre_trained_Model

pre_trained_model = CIFAR_Detectition_Pre_trained_Model('models/cifar_xception/', (96, 96))
cnn_model = CIFAR_Detectition_Pre_trained_Model('models/cifar_cnn_augmentation/', (32, 32))

**Compute predictions**

In [ ]:
y_pred_pre_trained, y_test_pre_trained, y_pred_classes_pre_trained = pre_trained_model.compute_predictions(ds_test)
y_pred_cnn, y_test_cnn, y_pred_classes_cnn = cnn_model.compute_predictions(ds_test)

### Compute confusion matrix

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
confusion_mtx_pre_trained = tf.math.confusion_matrix(y_test_pre_trained, y_pred_classes_pre_trained)
confusion_mtx_cnn = tf.math.confusion_matrix(y_test_cnn, y_pred_classes_cnn)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15, 15), sharex=True, sharey=True)
c_pre_trained = sns.heatmap(confusion_mtx_pre_trained, annot=True, fmt='g', ax=axs[0])
c_pre_trained.set(xticklabels=class_names, yticklabels=class_names)
c_pre_trained.set_title('Confusion matrix - Pre-trained model')

c_cnn = sns.heatmap(confusion_mtx_cnn, annot=True, fmt='g', ax=axs[1])
c_cnn.set(xticklabels=class_names, yticklabels=class_names)
c_cnn.set_title('Confusion matrix - CNN model')

### Make a report

**Pre-trained report**

In [ ]:
report = classification_report(y_test_pre_trained, y_pred_classes_pre_trained, target_names=class_names, output_dict=True)
df_pretrained = pd.DataFrame(report).transpose()
df_pretrained

**CNN model**

In [ ]:
report = classification_report(y_test_cnn, y_pred_classes_cnn, target_names=class_names, output_dict=True)
df_cnn = pd.DataFrame(report).transpose()
df_cnn

### Comparatie ROC-AUC Curve for each class

In [ ]:
def plot_ax(y_pred_pre_trained, y_test_pre_trained, y_pred_cnn, y_test_cnn, ax, i, label):

    fpr_pre_trained, tpr_pre_trained, thresh_pretrained = roc_curve(y_test_pre_trained, y_pred_pre_trained[:,i], pos_label=i)
    fpr_cnn, tpr_cnn, thresh_cnn = roc_curve(y_test_cnn, y_pred_cnn[:,i], pos_label=i)
    
    ax.plot(fpr_pre_trained, tpr_pre_trained, linestyle='--',color='orange', label='Pre-trained model')
    ax.plot(fpr_cnn, tpr_cnn, linestyle='--',color='green', label='CNN model')
    ax.set_title(label)
    ax.legend(loc='best')
    


In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(15, 15), sharex=True, sharey=True)

i = 0
for label in class_names:  
    
    plot_ax(y_pred_pre_trained, y_test_pre_trained, y_pred_cnn, y_test_cnn, axs[int(i/2), i%2], i, label)
    i += 1

fig.suptitle('AUC- ROC Curve One vs All')

# Set common labels
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')


plt.tight_layout()
plt.show()

## Cohen Kappa score

In [ ]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_test_pre_trained, y_pred_classes_pre_trained)

In [ ]:
cohen_kappa_score(y_test_cnn, y_pred_classes_cnn)